# Building a Database for Boston Crime Reports:  PostgreSQL and psycopg2 Module in Python
This project exemplifies the data engineering process in PostgreSQL to establish a relational database from csv file containing crime data in the Boston area.  The following processes are implemented:
- Database creation
- Schema creation
- Table creation with proper datatypes
- Loading data from csv file into database
- Group and user creation with appropriate priveleges

Here, data is loaded without using csv, instead of pandas, to illustrate that process.

## Importing Necessary Libraries

In [1]:
import psycopg2
import csv
import pandas as pd

## Create the Database

In [2]:
conn = psycopg2.connect(dbname='dq', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE DATABASE crime_db;')
conn.autocommit = False
conn.close()

## Create the Schema

In [3]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()

In [4]:
cur.execute('CREATE SCHEMA crimes;')

## Examining the Data

#### Checking the Header and First Row

In [5]:
with open('boston.csv', 'r') as f:
    reader = list(csv.reader(f))
    header_row = reader[0]
    first_row = reader[1:5]
    
print(header_row)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
[['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053'], ['2', '1402', 'VANDALISM', '2018-08-21', 'Tuesday', '42.30682138', '-71.06030035'], ['3', '3410', 'TOWED MOTOR VEHICLE', '2018-09-03', 'Monday', '42.34658879', '-71.07242943'], ['4', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.33418175', '-71.07866441']]


#### Observations:
There are 7 columns as follows:
- `incident_number`:  unique integer identifier
- `offense_code`:  integer representing offense
- `description`:  text/string description
- `date`: date of crime in ISO 8601 format
- `day_of_the_week`: weekday as string/text  
- `latitude`: float with precision of 10 and scale of 8
- `longitude`: float with precision of 10 and scale of 8

#### Function to Obtain Set of Distinct Values for a Column

In [6]:
def get_col_set(csv_file, col_num, header=True):
    """
    Returns set of distinct values for column of csv file
    
    csv_file: csv file name as string
    col_num: integer index of column
    header:  True or False if file has header row (default is True)
    """
    with open(csv_file, 'r') as f:
        rows = list(csv.reader(f))
        if header:
            rows = rows[1:]
        return {row[col_num] for row in rows}

#### Function to Obtain the Number of Unique Values in Dataset Columns

In [7]:
def get_num_unique(csv_file, header=True):
    '''
    Applies get_col_set function to all columns of csv file
    and returns dataframe of number of unique values in each column
    
    csv_file: csv file name as string
    header:  True or False if file has header row (default is True)
    '''
    num_unique = pd.DataFrame(columns=['num_unique'], index = header_row)    
    for i, value in enumerate(header_row):
        num_unique.loc[value, 'num_unique'] = len(get_col_set('boston.csv', i, header=header))
    return num_unique.reset_index().rename(columns={'index': 'column'})

#### Function to Obtain the Longest Value in Each Column

In [8]:
def longest_value(csv_file, header=True):
    '''
    Applies get_col_set function to all columns of csv file
    and returns dataframe of maximum value length for each column
    
    csv_file: csv file name as string
    header:  True or False if file has header row (default is True)    
    '''
    max_length = pd.DataFrame(columns=['max_length'], index=header_row)
    for i, value in enumerate(header_row):
        max_length.loc[value, 'max_length'] = max(len(value) for value in get_col_set(csv_file, i, header=header))
    return max_length.reset_index().rename(columns={'index': 'column'})

#### Checking the Number of Unique Values in Each Column

In [9]:
get_num_unique('boston.csv')

,column,num_unique
0,incident_number,298329
1,offense_code,219
2,description,239
3,date,1177
4,day_of_the_week,7
5,lat,18177
6,long,18177


#### Checking the Maximum Value Length for Each Column

In [10]:
longest_value('boston.csv')

,column,max_length
0,incident_number,6
1,offense_code,4
2,description,58
3,date,10
4,day_of_the_week,9
5,lat,11
6,long,12


#### Observations:
- We have 298,329 incidents in our dataset, that occurred on 1177 unique dates, with each weekday represented.
- The unique location information indicates that specific latitude and longitude information refers to a larger area, that mutliple events occurred at the same location, or that location data is missing for some entries.
- In our database table, we will typecast the columns as follows:
    - integer/int4: `incident_number`
    - smallint/int2: `offense_code`
    - enumerated: `day_of_the_week`
    - date: `date`
    - double precision/float8: `lat` and `long`
    - varchar(n): `description`

#### Creating Enumerated Types for `day_of_the_week` and `offense_code`

In [11]:
get_col_set('boston.csv', 4)

{'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'}

In [12]:
query = "CREATE TYPE enum_weekday AS ENUM ('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday');"
cur.execute(query)

### Creating Table `crimes.boston_crimes`

In [13]:
query = '''
CREATE TABLE crimes.boston_crimes (
    incident_number INT4 PRIMARY KEY,
    offense_code INT2,
    description VARCHAR(80),
    date DATE,
    day_of_the_week enum_weekday,
    lat FLOAT8,
    long FLOAT8
    );
'''
cur.execute(query)

#### Verifying Table Creation

In [14]:
query = 'SELECT * FROM crimes.boston_crimes LIMIT 0;'
cur.execute(query)
cur.description

(Column(name='incident_number', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='offense_code', type_code=21, display_size=None, internal_size=2, precision=None, scale=None, null_ok=None),
 Column(name='description', type_code=1043, display_size=None, internal_size=80, precision=None, scale=None, null_ok=None),
 Column(name='date', type_code=1082, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='day_of_the_week', type_code=16627, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='lat', type_code=701, display_size=None, internal_size=8, precision=None, scale=None, null_ok=None),
 Column(name='long', type_code=701, display_size=None, internal_size=8, precision=None, scale=None, null_ok=None))

#### Example of Checking `type_code`

In [15]:
query = 'SELECT typname FROM pg_catalog.pg_type WHERE oid = 1043;'
cur.execute(query)
cur.fetchone()[0]

'varchar'

## Loading Data from csv into PostgreSQL Table

In [16]:
with open('boston.csv', 'r') as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", f)

## Creating User Groups

#### Revoking `public` Privileges

In [17]:
query = "REVOKE ALL ON SCHEMA public FROM public;"
cur.execute(query)

query = "REVOKE ALL ON DATABASE crime_db FROM public;"
cur.execute(query)

#### Creating `readonly` and `readwrite` User Groups

In [18]:
query = "CREATE GROUP readonly NOLOGIN;"
cur.execute(query)

query = "CREATE GROUP readwrite NOLOGIN;"
cur.execute(query)

In [19]:
query = "GRANT CONNECT ON DATABASE crime_db TO readonly, readwrite;"
cur.execute(query)

query = 'GRANT USAGE ON SCHEMA crimes TO readonly, readwrite;'
cur.execute(query)

In [20]:
query = "GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;"
cur.execute(query)

query = "GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;"
cur.execute(query)

## Creating Individual Users

In [21]:
query = "CREATE USER data_analyst WITH PASSWORD 'secret1';"
cur.execute(query)

query = "GRANT readonly TO data_analyst;"
cur.execute(query)

In [22]:
query = "CREATE USER data_scientist WITH PASSWORD 'secret2';"
cur.execute(query)

query = "GRANT readwrite TO data_scientist;"
cur.execute(query)

## Committing SQL Transaction

In [23]:
conn.commit()

## Verifying Table Priveleges

In [24]:
query = '''
SELECT grantee, privilege_type
  FROM information_schema.table_privileges
 WHERE grantee IN ('readonly', 'readwrite');
 '''
cur.execute(query)
cur.fetchall()

[('readonly', 'SELECT'),
 ('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

## Test Queries

In [25]:
query = 'SELECT * FROM crimes.boston_crimes LIMIT 5;'
cur.execute(query)
cur.fetchall()

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  42.35779134,
  -71.13937053),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  42.30682138,
  -71.06030035),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  42.34658879,
  -71.07242943),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  42.33418175,
  -71.07866441),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  42.27536542,
  -71.09036101)]

In [26]:
query = "SELECT COUNT(*) FROM crimes.boston_crimes;"
cur.execute(query)
cur.fetchone()[0]

298329

#### Observations:
- The content of the first few rows is consistent with what we expected and the total number of rows matches that of the original csv file.
- We can commit and close our connection.

In [27]:
conn.commit()
conn.close()

## Appendix:  Code Cells for Resetting Notebook

In [28]:
# conn.rollback()
# conn.close()

In [29]:
# conn = psycopg2.connect(dbname='dq', user='dq')
# conn.autocommit=True
# cur = conn.cursor()
# cur.execute('DROP DATABASE crime_db;')

In [30]:
# conn = psycopg2.connect(dbname = 'dq', user='dq')
# cur = conn.cursor()

In [31]:
# cur.execute('DROP ROLE readonly;')

In [32]:
# cur.execute('DROP ROLE readwrite;')

In [33]:
# cur.execute('DROP ROLE data_scientist;')

In [34]:
# cur.execute('DROP ROLE data_analyst;')

In [35]:
# cur.execute('SELECT * FROM pg_group;')
# cur.fetchall()

In [36]:
# cur.execute('SELECT * FROM pg_user;')
# cur.fetchall()

In [37]:
# conn.commit()
# conn.close()